In [ ]:
import os
import sys
import os.path as op
import PIL
from PIL import Image
from fastai.vision import *
from fastai.vision.all import *
from typing import Union, List
import pandas as pd
import shutil

import uvicorn
from fastapi import File, UploadFile, FastAPI, status
from fastapi.encoders import jsonable_encoder

from fastapi.responses import JSONResponse

from modules import (
    DRModel,
    DRClassifier,
    ImageQualityModel,
    FundusImageQualityClassifier,
    YoloCupDiscSegmentor,
    GlaucomaClassifier,
    EyeScreener,
)
from modules.utils.segmentation_utils import *

## allow loading large images
from PIL import ImageFile

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

ImageFile.LOAD_TRUNCATED_IMAGES = True

TEMPORARY_IMAGE_FOLDER = "temp_images"
YOLO_CUP_DISC_SEGMENTOR_TEMP_DIR = "temp_predicted_masks"

image_quality_model_path = "trained_models/image_quality_resnet18_512d_512_18ep"
dr_model_path = "trained_models/dr_se_resnext50_32x4d_224_15ep"
cup_model_path = "trained_models/yolo-cup-15ep/weights/best.pt"
disc_model_path = "trained_models/yolo-disc-15ep/weights/best.pt"

In [ ]:
screener = EyeScreener(
    image_quality_model_path=image_quality_model_path,
    dr_model_path=dr_model_path,
    cup_model_path=cup_model_path,
    disc_model_path=disc_model_path,
    glaucoma_model_path=None,
    device=DEVICE,
    # Select the machine type (Nidek or Eidon).
    # machine_type="Nidek",
    # Optional kwargs for the cup_disc_segmentor and their default values.
    # measure_mask_length_from_height = True, # If False, then from width.
    temp_save_dir=YOLO_CUP_DISC_SEGMENTOR_TEMP_DIR,  # Change the name of the temp dir.
    clear_temp_dir_after=False,
)

In [ ]:
def predict(image):
    # Save the image to a temporary folder so that the screener can load it.
    temp_image_name = "temp_image.png"
    temp_image_path = op.join(TEMPORARY_IMAGE_FOLDER, temp_image_name)
    os.makedirs(TEMPORARY_IMAGE_FOLDER, exist_ok=True)
    image.save(temp_image_path)

    # Predict the image.
    predicted_dict = screener.predict(image_path=temp_image_path)
    # Parse the output.
    image_quality_output = {
        "Image Quality: Good": predicted_dict["image_quality"]["probability"]["good"],
        "Image Quality: Acceptable": predicted_dict["image_quality"]["probability"]["acceptable"],
        "Image Quality: Poor": predicted_dict["image_quality"]["probability"]["poor"],
    }

    return [image_quality_output]

In [ ]:
df = pd.read_csv("pred_img_quality_all_label_3000.csv")
# df = df[["filename", "original_path", "image_quality"]]
# df["original_path"] = df["original_path"].apply(lambda x: os.path.basename(x))
# df.dropna(inplace=True)
df

In [ ]:
df["pred_image_quality"]

In [ ]:
from lxml import etree
import pandas as pd
from tqdm import tqdm

xml_file = "../annotations.xml"
tree = etree.parse(xml_file)
images = tree.findall(".//image")

image_quality = []

for image in tqdm(images):
  image_name = image.get("name")
  tag = image.findall("tag")

  if tag is not None:
    for tg in tag:
      text = tg.get("label")

      if text == "GLAUCOMA SUSPECT":
        continue
  
      image_quality.append({"file_name": image_name,
                            "image_quality": text})
      

pd.DataFrame(image_quality).to_csv("image_quality_label.csv", index=False)

In [ ]:
df_pred = df[df["pred_image_quality"].isna()]
df_pred

In [ ]:
from glob import glob 

imgs = glob("../siriraj-eye-dataset-2023-jan/siriraj-eye-dataset-2023-jan/images/default/*.jpg")
original_paths = list(df_pred["file_name"])
print(len(imgs), len(original_paths))

In [ ]:
from tqdm.auto import tqdm

count = 0

for img in imgs:
    img_name = os.path.basename(img)
    if img_name in original_paths:
        count += 1
        # print(img_name)

print("Count: ", count, " / ", len(imgs))

In [ ]:
from tqdm.auto import tqdm

num = 1000

for img in tqdm(imgs):
    img_name = os.path.basename(img)

    if img_name in original_paths:
        results = predict(Image.open(img))
        quality = results[0]
        highest_quality = max(quality, key=quality.get)
        quality_type = highest_quality.split(': ')[1].upper()

        df.loc[df["file_name"] == img_name, "pred_image_quality"] = quality_type
        num += 1
        print(f"Working on .. images num {num}  .. :", img_name , end='\r')

    if num % 1000 == 0:
        df.to_csv(f"pred_img_quality_all_label_{num}.csv", index=False)

df = df.dropna()
df.to_csv(f"pred_img_quality_all_label_all.csv", index=False)

In [ ]:
new_df = df[df["image_quality"] != df["pred_image_quality"]]

new_df.to_csv("mismatched_img_quality.csv", index=False)

In [5]:
os.makedirs("dif_quality_pred_images", exist_ok=True)

df = pd.read_csv("mismatched_img_quality.csv")
df

,file_name,image_quality,pred_image_quality
0,603L (C).jpg,GOOD,ACCEPTABLE
1,160L (C).jpg,POOR,ACCEPTABLE
2,116R (2).jpg,ACCEPTABLE,GOOD
3,116L (2).jpg,ACCEPTABLE,POOR
4,106R (2).jpg,ACCEPTABLE,POOR
...,...,...,...
269,9e7b7717ab1aa7922eac8c27a860e500.jpg,GOOD,ACCEPTABLE
270,81300bb243f0558499b7019fd5357db0.jpg,ACCEPTABLE,GOOD
271,5efbecd76168a7859798532171747659.jpg,ACCEPTABLE,GOOD
272,5a5734d4024726606495d5f218a30011.jpg,POOR,ACCEPTABLE


In [6]:
imgs = glob("../siriraj-eye-dataset-2023-jan/siriraj-eye-dataset-2023-jan/images/default/*.jpg")

for img in imgs:
    if os.path.basename(img) in list(df["file_name"]):
        shutil.copy(img, "dif_quality_pred_images")

In [7]:
dif_imgs = glob("dif_quality_pred_images/*.jpg")
print(len(dif_imgs))

274


In [ ]:
dif = [df["image_quality"] == df["pred_image_quality"]]